 # Spaceship Titanic - Survival Prediction


In [ ]:

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


## Load Data

In [ ]:

a = pd.read_csv("train.csv")
t = pd.read_csv("test.csv")

a.head()


## Exploratory Data Analysis

In [ ]:

print("Shape:", a.shape)
print(a.info())
print(a.isnull().sum())

print("Target Distribution:")
print(a["Transported"].value_counts())


## Feature Engineering

In [ ]:

# Create total spending feature
spend_cols = ["RoomService","FoodCourt","ShoppingMall","Spa","VRDeck"]
a["TotalSpend"] = a[spend_cols].sum(axis=1)
t["TotalSpend"] = t[spend_cols].sum(axis=1)

# Extract deck from Cabin
a["Deck"] = a["Cabin"].str.split("/").str[0]
t["Deck"] = t["Cabin"].str.split("/").str[0]

a = a.drop(["Cabin","Name","PassengerId"], axis=1)
t_id = t["PassengerId"]
t = t.drop(["Cabin","Name","PassengerId"], axis=1)


## Preprocessing Pipeline

In [ ]:

y = a["Transported"]
a = a.drop("Transported", axis=1)

num = a.select_dtypes(include=["int64","float64"]).columns
cat = a.select_dtypes(include=["object","bool"]).columns

num_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="median")),
    ("scale", StandardScaler())
])

cat_pipe = Pipeline([
    ("impute", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

pre = ColumnTransformer([
    ("n", num_pipe, num),
    ("c", cat_pipe, cat)
])


## Train Baseline Model

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, stratify=y, random_state=42)

m = RandomForestClassifier(random_state=42)

pipe = Pipeline([
    ("pre", pre),
    ("model", m)
])

pipe.fit(X_train, y_train)
pred = pipe.predict(X_test)

print("Accuracy:", accuracy_score(y_test, pred))
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))


## Hyperparameter Tuning & Improvement

In [ ]:

param = {
    "model__n_estimators": [200, 400],
    "model__max_depth": [None, 10, 20],
    "model__min_samples_split": [2, 5]
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

g = GridSearchCV(pipe, param, cv=cv, scoring="accuracy", n_jobs=-1)
g.fit(a, y)

print("Best Params:", g.best_params_)
print("Best CV Accuracy:", g.best_score_)


## Train Final Model & Create Submission

In [ ]:

best = g.best_estimator_
best.fit(a, y)

pred_test = best.predict(t)

sub = pd.DataFrame({
    "PassengerId": t_id,
    "Transported": pred_test
})

sub.to_csv("submission.csv", index=False)
print("Submission file created!")
